# Canal bouillant - wall vaporisation

See canal.data for comments

In [ ]:
from trustutils import run

run.introduction("Yannick Gorsse & Elie Saikali")
run.TRUST_parameters()

In [ ]:
import os

run.reset()
run.initBuildDirectory()

dis = [ # ("VDF", "", "option_vdf { all_options }"), # not yet code : TODO FIXME Elie !!
        ("PolyMAC_P0", "Polyedriser dom", "option_polymac { }") ]

for d, dd, ddd in dis:
    run.addCaseFromTemplate("canal.data", f"{d}", {"dis" : d , "poly" : dd, "opts" : ddd})

run.printCases()
run.runCases()
run.tablePerf()

In [ ]:
from trustutils import plot

a=plot.Graph("Residus")
for d, _, _ in dis:
    a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/canal.dt_ev",label=f"{d}")

a.scale(yscale="log")

In [ ]:
for name, c in [
    ("TEMPERATURE_L", 0),
    ("PRESSION", 0),
    ("VITESSE_L", 2),
    ("VITESSE_G", 2),
    ("ALPHA", 0),
]:
    a = plot.Graph(name)
    for d, _, _ in dis:
        a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/canal_{name}_TRUST.son", compo=c, label=f"{d}")